In [1]:
import os
import cv2
import imutils
import numpy as np
from matplotlib import pyplot as plt
from classifier import Classifier
from helper import get_biggest_square_corners, get_structure_matrix
from helper import apply_filter_to_images, crop_the_images
from helper import break_the_images_in_81_tiles, rotate_the_images_to_upright

In [2]:
def read_images(path_to_images, number_of_images):
    images = list()

    
    for i in range(0, number_of_images):

        aux_file_name = ""
        if i < 9:
            aux_file_path = path_to_images + '0' +  str(i+1)  + '.jpg'
        else:
            aux_file_path = path_to_images + str(i+1) + '.jpg'

        aux_image = cv2.imread(aux_file_path)
        images.append(aux_image)


    return images


In [6]:
if __name__ == "__main__":
    # Just reading the images.
    number_of_images = 20
    path_to_images = './Tema1/antrenare/clasic/'
    clasic_sudoku_raw_images = read_images(path_to_images = path_to_images, number_of_images = number_of_images)

    # Prelucrating the images. Applying color change to gray, some Guasssian blur and otsu treshold.
    clasic_sudoku_prelucrated_images = apply_filter_to_images(images = clasic_sudoku_raw_images, 
                                    to_gray = True, blur_level = 3, sharpen = False, treshold = 'otsu')
    cv2.imwrite('task1.png', clasic_sudoku_prelucrated_images[5])
    input()
    
    # Using the corners of the contour of the biggest square in each image, I rotate the image
    #   until the upper corners are as alligned as possible. This should be enough to solve the tilted images problems.                                
    rotated_sudoku_prelucrated_images = rotate_the_images_to_upright(images = clasic_sudoku_prelucrated_images)

    # Again, using the corners of the contour of the biggest square in each image, I make a cut in the image,
    #   keeping only the area between the corners, in order to keep only the sudoku, 
    #   which is the part that I am interested in the most.
    #       offset - how many more pixels you want to delete from each margin. 
    #                   Using it to get rid (as much as possible) of the lines around the big sudoku square.
    cropped_sudoku_images = crop_the_images(images = rotated_sudoku_prelucrated_images, offset = 20)

    # If all past operations went well, here I break the image in 81 tiles. Each tile will represent 1/81 of the 
    #   image and it will corespond (if everything before went well) to a small square from the sudoku. 
    #       offset - how many more pixels you want to delete from each margin. 
    #                   Using it to get rid (as much as possible) of the lines around each small square.
    breaked_sudoku_images = break_the_images_in_81_tiles(cropped_sudoku_images, offset = 50)

    # =========================== TASK 1 - Extract each sudoku structure ===============================
    # Info: The result should be a matrix which contains 'o' for each empty small square in the sudoku 
    #           and 'x' for each occupied (by a number) small square in the sudoku.

    for i in range(number_of_images):
        # This function gets a list of small squares images and tries to see which ones have something in them
        #   and which not, based on how many white pixels each small square has inside of it. 
        #   After finishing it returns a the matrix required for this task.
        structure_matrix = get_structure_matrix(breaked_sudoku_images[i])
        # The rest of the code is only to write the results "as required" by the project.
        output = ""
        index = 0
        for each_line in structure_matrix:
            for each_tile in each_line:
                output += str(each_tile)
            index += 1
            if index % 3 == 0 and index != 27:
                output += "\n"

        with open("./fisiere_solutie/Gherasim_Rares_343/clasic/" + str(i+1) + "_predicted.txt", "w") as f:
            for i in range(len(output)):
                f.write(output[i])

    # ================ TASK 1 - BONUS - Extract each sudoku with numbers ===============================
    # Info: The result should be a matrix which contains 'o' for each empty small square in the sudoku 
    #           and the coresponding number for each ocupied small square in the sudoku.

    # This is a basic classifier that trains itself when initialized.
    # Basic svm.svc classifier with 'rbf' kernel and c set to 4.
    # The training images/labels are small tiles extracted from the 'clasic/antrenare' images and then
    #   classified by myself with numbers ranging from 1 to 9.
    clasiffier = Classifier()

    for i in range(20):
        # Using the structure matrix so that tiles that are represented with 'o' will not get into the classifier
        #   because they are already empty.
        structure_matrix = get_structure_matrix(breaked_sudoku_images[i])
        output = ""
        index = 0
        collumn = 0
        for line in range(len(structure_matrix)):
            for col in range(len(structure_matrix[line])):
                # I will process only the spaces where structure_matrix[line][col] != 'o'
                if structure_matrix[line][col] == 'o':
                    output += str(structure_matrix[line][col])
                else:
                    # Here I am giving the classifier my tile image and wait until it returns the 
                    #   prediction to me. Then, I add the prediction to the output.
                    try:
                        prediction = clasiffier.predict(breaked_sudoku_images[i][line//3][collumn])
                    except IndexError:
                        print(line, col)
                        input()
                    output += str(prediction[0])
                collumn += 1
        
            # The rest of the code is only to write the results "as required" by the project.

            index += 1

            if collumn == 9:
                collumn = 0
                
            if index % 3 == 0 and index != 27:
                output += "\n"

                
        with open("./fisiere_solutie/Gherasim_Rares_343/clasic/" + str(i+1) + "_bonus_predicted.txt", "w") as f:
            for i in range(len(output)):
                f.write(output[i])


KeyboardInterrupt: Interrupted by user